# Flight Price Prediction - EDA and Feature Engineering

In [88]:
# import basic libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [89]:
train_df = pd.read_excel("Flight Prediction/Data_Train.xlsx")
train_df.columns, train_df.shape

(Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
        'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
        'Additional_Info', 'Price'],
       dtype='object'),
 (10683, 11))

In [90]:
test_df = pd.read_excel("Flight Prediction/Test_set.xlsx")
test_df.columns, test_df.shape

(Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
        'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
        'Additional_Info'],
       dtype='object'),
 (2671, 10))

In [91]:
df = pd.concat([train_df,test_df],axis=0)

In [92]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.2+ MB


##### will split Date_of_Journey column into 3 columns Date, Month, Year

In [94]:
# df['Date']=df.Date_of_Journey.str.split('/').str[0].astype(int)
# df['Month']=df.Date_of_Journey.str.split('/').str[1].astype(int)
# df['Year']=df.Date_of_Journey.str.split('/').str[2].astype(int)

#OR

df['Date']=df.Date_of_Journey.apply(lambda x:x.split('/')[0])
df['Month']=df.Date_of_Journey.apply(lambda x:x.split('/')[1])
df['Year']=df.Date_of_Journey.apply(lambda x:x.split('/')[2])

df['Date']=df['Date'].astype(int)
df['Month']=df['Month'].astype(int)
df['Year']=df['Year'].astype(int)


In [95]:
df.drop(columns='Date_of_Journey',inplace=True)

In [96]:
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,9,6,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,5,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,1,3,2019


##### Arrival time : will take only time not Date is not required

In [97]:
df['arr_hour']=df.Arrival_Time.apply(lambda x:x.split(' ')[0].split(':')[0])
df['arr_min']=df.Arrival_Time.apply(lambda x:x.split(' ')[0].split(':')[1])

In [98]:
df['arr_hour']=df['arr_hour'].astype(int)
df['arr_min']=df['arr_min'].astype(int)

In [99]:
df.drop(columns='Arrival_Time',inplace=True)

In [100]:
df.head()

,Airline,Source,Destination,Route,Dep_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,arr_hour,arr_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,19h,2 stops,No info,13882.0,9,6,2019,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,5h 25m,1 stop,No info,6218.0,12,5,2019,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,4h 45m,1 stop,No info,13302.0,1,3,2019,21,35


##### Will apply same logic to departure time

In [101]:
df['Dep_Hour']=df['Dep_Time'].apply(lambda x:x.split(':')[0])
df['Dep_Min']=df['Dep_Time'].apply(lambda x:x.split(':')[1])

In [102]:
df['Dep_Hour']=df['Dep_Hour'].astype(int)
df['Dep_Min']=df['Dep_Min'].astype(int)

In [103]:
df.drop(columns='Dep_Time',inplace=True)

In [104]:
df.head()

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,arr_hour,arr_min,Dep_Hour,Dep_Min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882.0,9,6,2019,4,25,9,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218.0,12,5,2019,23,30,18,5
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302.0,1,3,2019,21,35,16,50


##### Missing values

In [105]:
df.isnull().sum()

Airline               0
Source                0
Destination           0
Route                 1
Duration              0
Total_Stops           1
Additional_Info       0
Price              2671
Date                  0
Month                 0
Year                  0
arr_hour              0
arr_min               0
Dep_Hour              0
Dep_Min               0
dtype: int64

In [106]:
df[df['Route'].isnull()]

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,arr_hour,arr_min,Dep_Hour,Dep_Min
9039,Air India,Delhi,Cochin,NaN,23h 40m,NaN,No info,7480.0,6,5,2019,9,25,9,45


In [107]:
df[(df['Source']=='Delhi') & (df['Destination']=='Cochin')].Total_Stops.value_counts()

1 stop      4015
2 stops     1373
non-stop     262
3 stops       31
Name: Total_Stops, dtype: int64

1. We have 1 missing value in Route and Total_Stops
2. From Delhi to Cochin we have all times of stops
3. Based on Duration=23h 40m we can take it 2 stops
4. So i am replacing with Total stops=3 for Null
5. even we can srop the record because we have only one record with null value

In [108]:
df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [109]:
df['Total_Stops']=df['Total_Stops'].map({'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, 'nan':3, '4 stops':4})

In [111]:
df['Total_Stops']=df['Total_Stops'].fillna(3)

In [113]:
df.drop(columns='Route',inplace=True)

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Duration         13354 non-null  object 
 4   Total_Stops      13354 non-null  float64
 5   Additional_Info  13354 non-null  object 
 6   Price            10683 non-null  float64
 7   Date             13354 non-null  int32  
 8   Month            13354 non-null  int32  
 9   Year             13354 non-null  int32  
 10  arr_hour         13354 non-null  int32  
 11  arr_min          13354 non-null  int32  
 12  Dep_Hour         13354 non-null  int32  
 13  Dep_Min          13354 non-null  int32  
dtypes: float64(2), int32(7), object(5)
memory usage: 1.2+ MB


1. Route and Total_stops give same information. will drop Route column

##### Will convert Hours and Minutes Duration column in single values minutes

In [115]:
df['Duration']

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
         ...   
2666    23h 55m
2667     2h 35m
2668     6h 35m
2669    15h 15m
2670    14h 20m
Name: Duration, Length: 13354, dtype: object

In [116]:
df['Duration_Hour']=df['Duration'].str.split(' ').str[0].str.split('h').str[0]

In [117]:
#df['Duration_Hour']=df['Duration_Hour'].astype(int)

In [118]:
df[df['Duration_Hour']=='5m']

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,arr_hour,arr_min,Dep_Hour,Dep_Min,Duration_Hour
6474,Air India,Mumbai,Hyderabad,5m,2.0,No info,17327.0,6,3,2019,16,55,16,50,5m
2660,Air India,Mumbai,Hyderabad,5m,2.0,No info,NaN,12,3,2019,16,55,16,50,5m


1. From Mumbai to Hyderabad duration is 5mins and Total_Stops is 2
2. The data seems like incorrect. So will drop these records based on index

In [119]:
df.shape

(13354, 15)

In [120]:
df.drop(index=[6474,2660],inplace=True)

In [121]:
df['Duration_Hour']=df['Duration_Hour'].astype(int)

In [122]:
df['Duration_Hour']=df['Duration_Hour']*60

In [123]:
df['Duration_min']=df['Duration'].str.split(' ').str[1].str.split('m').str[0]

In [124]:
df['Duration_min']=df['Duration_min'].fillna(0)

In [125]:
df[df['Duration_min'].isnull()]

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,arr_hour,arr_min,Dep_Hour,Dep_Min,Duration_Hour,Duration_min


In [126]:
df['Duration_min']=df['Duration_min'].astype(int)

In [127]:
df['Duration_new']=df['Duration_Hour']+df['Duration_min']

In [128]:
df['Duration_new']

0        170
1        445
2       1140
3        325
4        285
        ... 
2666    1435
2667     155
2668     395
2669     915
2670     860
Name: Duration_new, Length: 13351, dtype: int32

1. Droping newly created columns and Duration column also

In [129]:
df.drop(columns=['Duration_Hour','Duration_min','Duration'],inplace=True)

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13351 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13351 non-null  object 
 1   Source           13351 non-null  object 
 2   Destination      13351 non-null  object 
 3   Total_Stops      13351 non-null  float64
 4   Additional_Info  13351 non-null  object 
 5   Price            10681 non-null  float64
 6   Date             13351 non-null  int32  
 7   Month            13351 non-null  int32  
 8   Year             13351 non-null  int32  
 9   arr_hour         13351 non-null  int32  
 10  arr_min          13351 non-null  int32  
 11  Dep_Hour         13351 non-null  int32  
 12  Dep_Min          13351 non-null  int32  
 13  Duration_new     13351 non-null  int32  
dtypes: float64(2), int32(8), object(4)
memory usage: 1.1+ MB


In [131]:
df.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [132]:
df.Source.value_counts()

Delhi       5681
Kolkata     3581
Banglore    2752
Mumbai       881
Chennai      456
Name: Source, dtype: int64

In [133]:
df.Destination.value_counts()

Cochin       5681
Banglore     3581
Delhi        1582
New Delhi    1170
Hyderabad     881
Kolkata       456
Name: Destination, dtype: int64

In [134]:
df[['Source','Destination']].value_counts()

Source    Destination
Delhi     Cochin         5681
Kolkata   Banglore       3581
Banglore  Delhi          1582
          New Delhi      1170
Mumbai    Hyderabad       881
Chennai   Kolkata         456
dtype: int64

Here Destination contain both Delhi and New Delhi

In [135]:
df[df['Destination']=='New Delhi'].head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,arr_hour,arr_min,Dep_Hour,Dep_Min,Duration_new
0,IndiGo,Banglore,New Delhi,0.0,No info,3897.0,24,3,2019,1,10,22,20,170
4,IndiGo,Banglore,New Delhi,1.0,No info,13302.0,1,3,2019,21,35,16,50,285
6,Jet Airways,Banglore,New Delhi,1.0,In-flight meal not included,11087.0,12,3,2019,10,25,18,55,930
7,Jet Airways,Banglore,New Delhi,1.0,No info,22270.0,1,3,2019,5,5,8,0,1265
8,Jet Airways,Banglore,New Delhi,1.0,In-flight meal not included,11087.0,12,3,2019,10,25,8,55,1530


In [136]:
df.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [137]:
df.Additional_Info.replace('No Info','No info',inplace=True)

In [138]:
df.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

### Lets perform One hot encoding or Getdummy on Airline, Source, Destination and Additional_info

In [139]:
df=pd.get_dummies(data=df,columns=['Airline','Source','Destination','Additional_Info'],drop_first=True)

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13351 entries, 0 to 2670
Data columns (total 38 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Total_Stops                                   13351 non-null  float64
 1   Price                                         10681 non-null  float64
 2   Date                                          13351 non-null  int32  
 3   Month                                         13351 non-null  int32  
 4   Year                                          13351 non-null  int32  
 5   arr_hour                                      13351 non-null  int32  
 6   arr_min                                       13351 non-null  int32  
 7   Dep_Hour                                      13351 non-null  int32  
 8   Dep_Min                                       13351 non-null  int32  
 9   Duration_new                                  13351 non-null  

In [141]:
train=df[~df['Price'].isnull()]
test=df[df['Price'].isnull()]

train.shape, test.shape

((10681, 38), (2670, 38))

In [142]:
X=train.loc[:,train.columns != 'Price']
y=train['Price']
X.shape, y.shape

((10681, 37), (10681,))

In [143]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split(X,y)

In [144]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8010, 37), (2671, 37), (8010,), (2671,))

In [147]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [148]:
y_predict = lr.predict(X_test)
y_predict

array([10009.20045211,  4096.99347619,  4578.00061091, ...,
       12973.56724702,  9924.02273733, 10761.24378695])

In [150]:
w = lr.coef_
b = lr.intercept_
w,b

(array([ 2.46357961e+03, -6.52522846e+01, -3.82680239e+02,  6.08961273e-08,
        -1.45652123e+01,  2.51978973e+00,  2.12565376e+01, -2.34156795e+00,
         1.79877216e-01,  1.60283211e+03,  1.11312471e+02,  1.34193217e+02,
         6.11049048e+03,  4.19807937e+04,  3.91520274e+03,  3.54574039e+03,
        -2.73272142e+02, -2.29600400e+03,  2.11166799e+03,  2.93838650e+03,
        -5.83419927e+00,  1.40729117e+02,  7.68058281e+01, -8.67694472e+02,
         1.40729117e+02, -9.49124054e+02, -8.67694472e+02, -5.83419927e+00,
         1.60511778e+03,  4.16379693e+03,  0.00000000e+00,  1.12345192e+04,
        -7.04542846e+03, -1.22255286e+04, -8.82840211e+03, -8.58871348e+03,
        -5.69395879e+03]),
 15719.02550805383)

In [153]:
lr.score(X_test,y_test)

0.7194050388112927